- 47%
- Missing data files

In [1]:
%%bash
# Do not change or modify this file
# Need to install pyspark
# if pyspark is already installed, will print a message indicating pyspark already isntalled
pip install pyspark


  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=e207dfb7ee04ca86210a6f9028d6e6838eb3fa5cdb43947c3822d690ccfec011
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
spark = SparkSession.builder\
  .master("local[*]")\
  .config("spark.memory.fraction", 0.8)\
  .config("spark.executor.memory", "12g")\
  .config("spark.driver.memory", "12g")\
  .config("spark.memory.offHeap.enabled",'true')\
  .config("spark.memory.offHeap.size","12g")\
  .getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [3]:
from pyspark.sql.functions import isnan, isnull, when, count, col
import pandas as pd
from pyspark.sql.functions import col
import pyspark.sql.functions as fn
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql.functions import isnan, isnull, when, count, col

In [4]:
data_df = spark.read.option("header",True).csv("data.csv")
artist_df = spark.read.option("header",True).csv("data_by_artist.csv")
genres_df = spark.read.option("header",True).csv("data_by_genres.csv")
year_df = spark.read.option("header",True).csv("data_by_year.csv") 
wgenres_df = spark.read.option("header",True).csv("data_w_genres.csv") 

AnalysisException: ignored

ANALYSIS AND DATA CLEANING of DATA.CSV

In [ ]:
display(data_df.toPandas().head())

In [ ]:

data_df.select([count(when(isnull(c), c)).alias(c) for c in data_df.columns]).show()
data_df.select([count(when(isnan(c), c)).alias(c) for c in data_df.columns]).show()


In [ ]:
data_df = data_df.dropDuplicates()
types = [fn.dataType for fn in data_df.schema.fields]
types

#converting the duration in mins
data_df=data_df.withColumn("duration_min", (col("duration_ms") /60000))
data_df=data_df.withColumn("duration_min", fn.round(data_df["duration_min"], 2))
data_df = data_df.drop("id","release_date","duration_ms")


In [ ]:
print((data_df.count(), len(data_df.columns)))

In [ ]:
#changing the datatype
data_df = data_df.withColumn("acousticness",data_df["acousticness"].cast("float"))
data_df = data_df.withColumn("danceability",data_df["danceability"].cast("float"))
data_df = data_df.withColumn("energy",data_df["energy"].cast("float"))
data_df = data_df.withColumn("explicit",data_df["explicit"].cast("float"))
data_df = data_df.withColumn("instrumentalness",data_df["instrumentalness"].cast("float"))
data_df = data_df.withColumn("key",data_df["key"].cast("float"))
data_df = data_df.withColumn("liveness",data_df["liveness"].cast("float"))
data_df = data_df.withColumn("loudness",data_df["loudness"].cast("float"))
data_df = data_df.withColumn("mode",data_df["mode"].cast("float"))
data_df = data_df.withColumn("popularity",data_df["popularity"].cast("float"))
data_df = data_df.withColumn("speechiness",data_df["speechiness"].cast("float"))
data_df = data_df.withColumn("tempo",data_df["tempo"].cast("float"))
data_df = data_df.withColumn("valence",data_df["valence"].cast("float"))
data_df = data_df.withColumn("year",data_df["year"].cast("float"))
data_df = data_df.withColumn("duration_min",data_df["duration_min"].cast("float"))

TIME ANALYSIS

DATA ANALYSIS AND DATA CLEANING OF DATA_BY_ARTIST.CSV


In [ ]:
display(artist_df.toPandas().head())

In [ ]:
artist_df.select([count(when(isnull(c), c)).alias(c) for c in artist_df.columns]).show()
artist_df.select([count(when(isnan(c), c)).alias(c) for c in artist_df.columns]).show()

In [ ]:
artist_df.describe().show()

In [ ]:
artist_df=artist_df.withColumn("duration_min", (col("duration_ms") /60000))
artist_df=artist_df.withColumn("duration_min", fn.round(artist_df["duration_min"], 2))
artist_df = artist_df.drop("duration_ms")



In [ ]:
artist_df = artist_df.dropDuplicates()
types = [fn.dataType for fn in artist_df.schema.fields]
types

In [ ]:
artist_df = artist_df.withColumn("acousticness",artist_df["acousticness"].cast("float"))
artist_df = artist_df.withColumn("danceability",artist_df["danceability"].cast("float"))
artist_df = artist_df.withColumn("energy",artist_df["energy"].cast("float"))
artist_df = artist_df.withColumn("instrumentalness",artist_df["instrumentalness"].cast("float"))
artist_df = artist_df.withColumn("key",artist_df["key"].cast("float"))
artist_df = artist_df.withColumn("liveness",artist_df["liveness"].cast("float"))
artist_df = artist_df.withColumn("loudness",artist_df["loudness"].cast("float"))
artist_df = artist_df.withColumn("mode",artist_df["mode"].cast("float"))
artist_df = artist_df.withColumn("popularity",artist_df["popularity"].cast("float"))
artist_df = artist_df.withColumn("speechiness",artist_df["speechiness"].cast("float"))
artist_df = artist_df.withColumn("tempo",artist_df["tempo"].cast("float"))
artist_df = artist_df.withColumn("valence",artist_df["valence"].cast("float"))
artist_df = artist_df.withColumn("count",artist_df["count"].cast("float"))
artist_df = artist_df.withColumn("duration_min",artist_df["duration_min"].cast("float"))

ANALYSIS AND DATA CLEANING ON GENRE.CSV

In [ ]:
display(genres_df.toPandas().head())

In [ ]:
#removing the invalid data
genres_df = genres_df.withColumn('genres', when(col('genres') == '[]', None).otherwise(col('genres')))
genres_df = genres_df.withColumn('genres', when(col('genres') == '432hz', None).otherwise(col('genres')))

genres_df.select([count(when(isnull(c), c)).alias(c) for c in genres_df.columns]).show()
genres_df.select([count(when(isnan(c), c)).alias(c) for c in genres_df.columns]).show()

genres_df = genres_df.dropna(subset=['genres'])

In [ ]:
genres_df=genres_df.withColumn("duration_min", (col("duration_ms") /60000))
genres_df=genres_df.withColumn("duration_min", fn.round(genres_df["duration_min"], 2))
genres_df = genres_df.drop("duration_ms")


In [ ]:
genres_df = genres_df.dropDuplicates()
types = [fn.dataType for fn in genres_df.schema.fields]
types

In [ ]:
genres_df = genres_df.withColumn("acousticness",genres_df["acousticness"].cast("float"))
genres_df = genres_df.withColumn("danceability",genres_df["danceability"].cast("float"))
genres_df = genres_df.withColumn("energy",genres_df["energy"].cast("float"))
genres_df = genres_df.withColumn("instrumentalness",genres_df["instrumentalness"].cast("float"))
genres_df = genres_df.withColumn("liveness",genres_df["liveness"].cast("float"))
genres_df = genres_df.withColumn("loudness",genres_df["loudness"].cast("float"))
genres_df = genres_df.withColumn("mode",genres_df["mode"].cast("float"))
genres_df = genres_df.withColumn("popularity",genres_df["popularity"].cast("float"))
genres_df = genres_df.withColumn("speechiness",genres_df["speechiness"].cast("float"))
genres_df = genres_df.withColumn("tempo",genres_df["tempo"].cast("float"))
genres_df = genres_df.withColumn("valence",genres_df["valence"].cast("float"))
genres_df = genres_df.withColumn("duration_min",genres_df["duration_min"].cast("float"))

ANALYSIS AND DATA CLEANING ON YEAR.CSV

In [ ]:
display(year_df.toPandas().head())
year_df.select([count(when(isnull(c), c)).alias(c) for c in year_df.columns]).show()
year_df.select([count(when(isnan(c), c)).alias(c) for c in year_df.columns]).show()
year_df=year_df.withColumn("duration_min", (col("duration_ms") /60000))
year_df=year_df.withColumn("duration_min", fn.round(year_df["duration_min"], 2))
year_df = year_df.drop("duration_ms")

In [ ]:
year_df = year_df.dropDuplicates()
types = [fn.dataType for fn in year_df.schema.fields]
types

In [ ]:
year_df = year_df.withColumn("acousticness",year_df["acousticness"].cast("float"))
year_df = year_df.withColumn("danceability",year_df["danceability"].cast("float"))
year_df = year_df.withColumn("energy",year_df["energy"].cast("float"))
year_df = year_df.withColumn("instrumentalness",year_df["instrumentalness"].cast("float"))
year_df = year_df.withColumn("liveness",year_df["liveness"].cast("float"))
year_df = year_df.withColumn("loudness",year_df["loudness"].cast("float"))
year_df = year_df.withColumn("mode",year_df["mode"].cast("float"))
year_df = year_df.withColumn("popularity",year_df["popularity"].cast("float"))
year_df = year_df.withColumn("speechiness",year_df["speechiness"].cast("float"))
year_df = year_df.withColumn("tempo",year_df["tempo"].cast("float"))
year_df = year_df.withColumn("valence",year_df["valence"].cast("float"))
year_df = year_df.withColumn("year",year_df["year"].cast("float"))
year_df = year_df.withColumn("duration_min",year_df["duration_min"].cast("float"))

ANALYSIS AND DATA CLEANING WGENRE.CSV

In [ ]:
display(wgenres_df.toPandas().head())

In [ ]:
wgenres_df.select([count(when(isnull(c), c)).alias(c) for c in wgenres_df.columns]).show()
wgenres_df.select([count(when(isnan(c), c)).alias(c) for c in wgenres_df.columns]).show()

In [ ]:
#wgenres_df=wgenres_df.withColumn("duration_min", (col("duration_ms") /60000))
#wgenres_df=wgenres_df.withColumn("duration_min", fn.round(wgenres_df["duration_min"], 2))
wgenres_df = wgenres_df.drop("duration_ms")

In [ ]:
#more than 30% of data has genre as [], imputing them with mode value will deviate the data so we have removed those values
wgenres_df = wgenres_df.withColumn('genres', when(col('genres') == '[]', None).otherwise(col('genres')))
wgenres_df = wgenres_df.dropna(subset=['genres'])

In [ ]:
wgenres_df = wgenres_df.dropDuplicates()
types = [fn.dataType for fn in wgenres_df.schema.fields]
types

In [ ]:
wgenres_df = wgenres_df.withColumn("acousticness",wgenres_df["acousticness"].cast("float"))
wgenres_df = wgenres_df.withColumn("danceability",wgenres_df["danceability"].cast("float"))
wgenres_df = wgenres_df.withColumn("energy",wgenres_df["energy"].cast("float"))
wgenres_df = wgenres_df.withColumn("instrumentalness",wgenres_df["instrumentalness"].cast("float"))
wgenres_df = wgenres_df.withColumn("liveness",wgenres_df["liveness"].cast("float"))
wgenres_df = wgenres_df.withColumn("loudness",wgenres_df["loudness"].cast("float"))
wgenres_df = wgenres_df.withColumn("mode",wgenres_df["mode"].cast("float"))
wgenres_df = wgenres_df.withColumn("popularity",wgenres_df["popularity"].cast("float"))
wgenres_df = wgenres_df.withColumn("speechiness",wgenres_df["speechiness"].cast("float"))
wgenres_df = wgenres_df.withColumn("tempo",wgenres_df["tempo"].cast("float"))
wgenres_df = wgenres_df.withColumn("valence",wgenres_df["valence"].cast("float"))
wgenres_df = wgenres_df.withColumn("count",wgenres_df["count"].cast("float"))
#wgenres_df = wgenres_df.withColumn("duration_min",wgenres_df["duration_min"].cast("float"))

DATA VISUALIZATION:

In [ ]:
plt.figure(figsize=(16, 8))
sns.set(style="whitegrid")
corr = data_df.toPandas().corr()
sns.heatmap(corr,annot=True,cmap="coolwarm")

In [ ]:
plt.figure(figsize=(16, 8))
sns.set(style="whitegrid")
cols = ["valence","popularity","acousticness","instrumentalness","speechiness","danceability" ]
sns.pairplot(genres_df.toPandas()[cols], height = 2.5)
plt.show();



In [ ]:

x=genres_df.toPandas().sort_values('popularity').tail(10)
fig = plt.figure(figsize=(15,4))
plt.bar( x['genres'],
         x['popularity'],
         width=0.5,
         color = ['brown','gold'])
plt.title('Genres VS Popularity',y=1.1,fontsize=20)
plt.xlabel('Genres',fontsize=20)
plt.ylabel('Popularity',fontsize=20)
plt.xticks(fontsize=20,rotation=45,ha='right')
#x.axes.get_xaxis().set_visible(True)

In [ ]:
data = genres_df.select("genres")
data = data.toPandas()
fig = plt.figure(figsize = (10, 10))
ax = fig.subplots()
data.value_counts()[:10].plot(ax=ax, kind = "pie")
ax.set_ylabel("")
ax.set_title("Top 10 most popular genres")
plt.show()

In [ ]:
plt.figure(figsize=(16, 8))
sns.set(style="whitegrid")
x = data_df.toPandas().groupby("artists")["popularity"].sum().sort_values(ascending=False).head(10)
ax = sns.barplot(x,x.index)
ax.set_title('Top Artists with Popularity')
ax.set_ylabel('Artists')
ax.set_xlabel('Popularity')
plt.xticks(rotation = 90)

In [ ]:
data = data_df.toPandas()
x = data.groupby("loudness")["popularity"].mean().sort_values(ascending=False).head(20).reset_index()

plt.figure(figsize=(12,8))
sns.pointplot(x["loudness"].values, x['popularity'].values)
plt.ylabel('loudness', fontsize=12)
plt.xlabel('popularity', fontsize=12)
plt.title("popularity wise loudness", fontsize=15)
plt.xticks(rotation='vertical')
plt.show()

TIME ANALYSIS

In [ ]:
plt.figure(figsize=(16, 8))
sns.set(style="whitegrid")
x = data_df.toPandas().groupby("year")["name"].count()
ax= sns.lineplot(x.index,x)
ax.set_title('Count of Tracks added')
plt.xlim(1920,2020)
ax.set_ylabel('Count')
ax.set_xlabel('Year')

In [ ]:
plt.figure(figsize=(16, 8))
sns.set(style="whitegrid")
columns = ["acousticness","danceability","speechiness","liveness","valence"]
for col in columns:
    x = data_df.toPandas().groupby("year")[col].mean()
    ax= sns.lineplot(x=x.index,y=x,label=col)
ax.set_title('Audio characteristics over year')
plt.xlim(1920,2020)
plt.ylim(0,1)
ax.set_ylabel('Measure')
ax.set_xlabel('Year')

In [ ]:
plt.figure(figsize=(16, 8))
sns.set(style="whitegrid")
columns = ["tempo"]
for col in columns:
    x = data_df.toPandas().groupby("year")[col].mean()
    ax= sns.lineplot(x=x.index,y=x,label=col)
ax.set_title('tempo')
plt.xlim(1920,2020)
plt.ylim(100,125)
ax.set_ylabel('Count')
ax.set_xlabel('Year')

In [ ]:
data1_df= data_df.na.drop(how='any')
data1_df
data1_df.show()

In [ ]:
types = [fn.dataType for fn in data1_df.schema.fields]
types

In [ ]:
columns_drop= ['artists','name']
data2_df= data1_df.drop(*columns_drop)
data2_df= data2_df.withColumnRenamed('popularity','label')
data2_df.show()

**Linear Regression**

In [ ]:
#vc = VectorAssembler().setInputCols(train.columns[2:]).setOutputCol('f
from pyspark.ml.regression import LinearRegression
from pyspark.ml import feature
from pyspark.ml import clustering
from pyspark.ml import Pipeline
from pyspark.sql import functions as fn
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
import requests
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer
#Splitting the above transformed data into train and test (75% and 25% respectively)
train,test = data2_df.randomSplit([0.75,0.25],seed=40)

vc = feature.VectorAssembler(inputCols=['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence'], outputCol='features')
lin_reg = LinearRegression(featuresCol='features', labelCol='label')
pipe_lr = Pipeline(stages=[vc, lin_reg])
model1 = pipe_lr.fit(train)
#transforming the data 
model1.transform(test)
trainingSummary = model1.stages[-1].summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)




In [ ]:
from pyspark.sql.functions import col
len(train.columns)



In [ ]:
test.count()
len(test.columns)

**Random Forest Regression**

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4)
# Split the data into training and test sets (30% held out for testing)

# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="indexedFeatures")

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[vc,featureIndexer, rf])

model = pipeline.fit(train)
# Make predictions.
predictions = model.transform(test)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

rfModel = model.stages[1]
print(rfModel)  # summary only



**Mean Squared Error of Linear Regression Train/Test Outputs**

In [ ]:


train_output = model1.transform(train)
test_output = model1.transform(test)
#Making use of RegressionEvaluator for predicting the mean square error with the dependent variable as price 
linear_evaluator = RegressionEvaluator(predictionCol="prediction",metricName="mse",labelCol='label')

print('Mean Squared Error for Train data= ',linear_evaluator.evaluate(train_output))
print('Mean Squared Error for Test data= ',linear_evaluator.evaluate(test_output))

**Cross-Validation**

In [ ]:
from pyspark.ml.tuning import  CrossValidator,ParamGridBuilder

parmbuild = (ParamGridBuilder().build())
crossvalidation = CrossValidator(estimator=pipe_lr,                                     #Applying Cross Validator for evaluating machine learning model
                    estimatorParamMaps=parmbuild,
                    evaluator=linear_evaluator, numFolds=3,seed=20)                     #3 folds as per the question
cv_model = crossvalidation.fit(train)                                                        #Applyinh cross validation on train data 
avg_train= cv_model.avgMetrics                                                                     #Calculating the average metrics

print("The Average metrics is ",avg_train)


**Features of songs affecting popularity using Linear Regression**

In [ ]:
scalar = StandardScaler(inputCol = 'features', outputCol='scaled_features')  
inference_pipe = Pipeline(stages=[vc,scalar,lin_reg])   
b = inference_pipe.fit(train)
value=b.stages[-1].coefficients                
coefficient=b.stages[0].getInputCols()       
coeff_df=pd.DataFrame(coefficient,columns=['features'])   
coeff_df.insert(1,"value",value,True) 
display(coeff_df)

**Features of songs affecting popularity using Random Forest Regression**

In [ ]:
scalar = StandardScaler(inputCol = 'indexedFeatures', outputCol='scaled_features')  
raf = RandomForestRegressor(featuresCol="scaled_features")

in_pipe = Pipeline(stages=[vc,featureIndexer,scalar,raf])   
c = in_pipe.fit(train)
#value=c.stages[-1].coefficients                
coefficient=c.stages[0].getInputCols()       
coefficients_df=pd.DataFrame(coefficient,columns=['features'])   
coefficients_df.insert(1,"value",value,True) 
display(coefficients_df)

**PCA**

In [ ]:

vector_assemb = feature.VectorAssembler(inputCols=['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence'], outputCol='features')


std_scaler= feature.StandardScaler(withMean=True,withStd=False,inputCol='features', outputCol='scaled_features')

#norm = feature.Normalizer(inputCol="scaled_features", outputCol="norm_features", p=2.0)

#kmeans = clustering.KMeans(k=5, featuresCol='norm_features', predictionCol='kmeans_feat')

#PCA 
pca= feature.PCA(k=10,inputCol='scaled_features', outputCol='scores')

#Pipelining
pipe_pca_1 = Pipeline(stages=[vector_assemb,std_scaler,pca])
fit_model= pipe_pca_1.fit(data1_df)
chars_pca_df= fit_model.transform(data1_df)

In [ ]:
chars_pca_df.show()

**Recommending Songs**

In [ ]:
import numpy as np

def get_euclidian(c1, c2):
      return float(np.sqrt((c1 - c2).T.dot((c1 - c2))))   
l2_dist_udf = fn.udf(get_euclidian)
spark.sql("SET spark.sql.crossJoin.enabled=TRUE").show()

def get_nearest_chars(char,num_nearest_char):
        chars_pca_df.where(fn.col('name')==char).select(fn.col("scores").alias('pca_scores')).join(fit_model.transform(data1_df)).\
        withColumn('dist', l2_dist_udf('scores', 'pca_scores')).\
        select("name","artists","dist").\
        orderBy(fn.asc("dist")).\
        show(10, 80) 



In [ ]:
print((get_nearest_chars('Well Done',5)))

**Recommending Artists**

In [ ]:
import numpy as np


vector_assem = feature.VectorAssembler(inputCols=['acousticness', 'danceability','duration_min','energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence','popularity','mode','count'], outputCol='art_features')
std_scale= feature.StandardScaler(withMean=True,withStd=False,inputCol='art_features', outputCol='scale_features')
artist_pca= feature.PCA(k=3, inputCol='scale_features', outputCol='art_scores')
pipe_pca_2 = Pipeline(stages=[vector_assem,std_scale,artist_pca])
artist_model= pipe_pca_2.fit(artist_df)
artist_pca_df= artist_model.transform(artist_df)

def get_euclidian2(c1, c2):
      return float(np.sqrt((c1 - c2).T.dot((c1 - c2))))   
l3_dist_udf = fn.udf(get_euclidian2)
spark.sql("SET spark.sql.crossJoin.enabled=TRUE").show()

def get_nearest_artist(char,num_nearest_char):
        artist_pca_df.where(fn.col('artists')==char).select(fn.col("art_scores").alias('artist_pca_scores')).join(artist_model.transform(artist_df)).\
        withColumn('dist', l3_dist_udf('art_scores', 'artist_pca_scores')).\
        select("artists","dist").\
        orderBy(fn.asc("dist")).\
        show(10, 80) 

In [ ]:
print((get_nearest_artist('John Legend',5)))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.figure()
explained_var = fit_model.stages[-1].explainedVariance
plt.plot(np.arange(1, len(explained_var)+1), explained_var)
plt.title("Scree Plot")
plt.xlabel("Principal Component")
plt.ylabel("Proportion Variance Explained")
cum_sum = np.cumsum(explained_var)


In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

silhouette_scores = []


kmeans = KMeans(featuresCol='scores')
evaluator = ClusteringEvaluator(featuresCol='scores')
for k in range(2,7):
  fit_k = kmeans.setK(k).fit(chars_pca_df)
  predictions= fit_k.transform(chars_pca_df)
  silhouette = evaluator.evaluate(predictions)
  print("Silhouette score"+ str(silhouette),k)
  silhouette_scores.append([k,silhouette])

centers = fit_k.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

In [ ]:
plt.plot(*zip(*silhouette_scores))
plt.xlabel('k',fontsize=14)
plt.ylabel('silhouette scores',fontsize=14)
plt.title('silhoutte scores vs k',y=1)

In [ ]:
from pyspark.ml.functions import vector_to_array as vca

df_kmeans = kmeans.setK(4).fit(chars_pca_df).transform(chars_pca_df)
silhouette = evaluator.evaluate(df_kmeans)
print("Silhouette score = " + str(silhouette))
df_kmeans.limit(10).toPandas()
predict_df = df_kmeans.withColumn('scores',vca('scores')).select([fn.col('scores')[0],fn.col('scores')[1],'prediction']).toPandas()

fig, ax = plt.subplots(1,1, figsize =(8,6))
scatter= ax.scatter(predict_df['scores[0]'],predict_df['scores[1]'] ,c = predict_df.prediction,cmap = 'viridis',alpha=0.5)
plt.title('PC1 vs PC2',y=1)
plt.xlabel('PC1',fontsize=14)
plt.ylabel('PC2',fontsize=14)
ax.legend(*scatter.legend_elements(), loc='upper left', title='Kth cluster')